# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
from datetime import timedelta
import papermill as pm
from nord_h2ub_dropdown_functions import *

### Base Parameters:

In [2]:
# Example usage
combined_tabs, dropdowns = create_combined_dropdowns_tabs()

In [5]:
#get the values after setting the inputs by the user
values = get_dropdown_values(dropdowns)

In [ ]:
#define parameters for the use of the roll forward (rolling horizon) approach
#set the number of slices within the modelling horizon
roll_forward_use = True
num_slices = 12
# Create DatetimeIndex for the range of dates
start_date = pd.Timestamp(str(values['year']) + '-01-01 00:00:00')
end_date = pd.Timestamp(str(values['year']) + '-12-31 23:00:00')
datetime_index = pd.date_range(start=start_date, end=end_date, freq=frequency_dropdown.value)
# Calculate the number of steps within the horizon
num_steps = len(datetime_index)
# Check if the number of slices can be used
# Find the largest integer divisor that fulfils the condition
for i in range(num_slices, 0, -1):
    if num_steps % i == 0:
        roll_forward_size = num_steps // i
        used_slices = i
        break
else:
    print("Cannot divide the number of steps into any integer slices. Please choose a different number of slices.")
    exit()
# Check if num_slices matches the used_slices
if num_slices != used_slices:
    print("\033[91mWARNING:\033[0m The specified number of slices (", num_slices, ")",
          "does not match the final division factor (", used_slices, ").",
         "\n The calculation uses the factor: ",used_slices, ".")


# Print warning message in red
print("\033[91mWARNING:\033[0m Please control if all the parameters are set correctly")

In [4]:
#temporal block, do not change
if dropdowns['frequency'].value == '1h':
    temporal_block = 'hourly'
elif dropdowns['frequency'].value == '1D':
    temporal_block = 'daily'
elif dropdowns['frequency'].value == '1W':
    temporal_block = 'weekly'
elif dropdowns['frequency'].value == '1M':
    temporal_block = 'monthly'
elif dropdowns['frequency'].value == '1Q':
    temporal_block = 'quarterly'
else: 
    temporal_block = 'yearly'

### Dictionary:

In [6]:
base_parameters = {
    'year': values['year'],
    'start_date': (pd.Timestamp(str(values['year']) + '-01-01 00:00:00')).strftime('%Y-%m-%d %H:%M:%S'),
    'end_date': (pd.Timestamp(str(values['year']) + '-12-31 23:00:00')).strftime('%Y-%m-%d %H:%M:%S'),
    'area': values['price_zone'],
    'product': values['product'],
    'scenario': [values['base_scen']] + values['other_scen'].split('\n'),
    'frequency': values['frequency'],
    'model_name': values['model_name'],
    'temporal_block': temporal_block,
    'stochastic_scenario': values['stoch_scen'],
    'stochastic_structure': values['stoch_struc'],
    'report_name': values['report_name'],
    'reports': values['outputs'],
    'electrolyzer_type': values['electrolysis'],
    'des_segments_electrolyzer': des_segments_electrolyzer,
    'share_of_dh_price_cap': values['share_of_dh_price_cap'],
    'price_level_power': values['number_price_level_power'],
    'power_price_variance': values['power_price_variance'],
    'roll_forward_use': roll_forward_use,
    'roll_forward_size': roll_forward_size,
    'num_slices': num_slices,
    'num_steps': num_steps
}

DateParseError: Unknown datetime string format, unable to parse: None-01-01 00:00:00

### Execute standard routine:

In [ ]:
pm.execute_notebook(
    'data_preparation_nord_h2ub_spine.ipynb',
    'please_ignore.ipynb',
    parameters=base_parameters
)